In [2]:
import wfdb
from FeatureExtraction_NB import processing,process_signal,process_qrs
import matplotlib.pyplot as plt
import pywt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
fs = 1000

patient_1 = wfdb.rdrecord('..//01.Dataset/117/s0291lre', channels=[1])
patient_2 = wfdb.rdrecord('..//01.Dataset/116/s0302lre', channels=[1])
patient_3 = wfdb.rdrecord('..//01.Dataset/105/s0303lre', channels=[1])
patient_4 = wfdb.rdrecord('..//01.Dataset/104/s0306lre', channels=[1])
patient_5 = wfdb.rdrecord('..//01.Dataset/122/s0312lre', channels=[1])
patient_6 = wfdb.rdrecord('..//01.Dataset/121/s0311lre', channels=[1])
patient_7 = wfdb.rdrecord('..//01.Dataset/150/s0287lre', channels=[1])
 

print(patient_1)


signal_1,_,_ = processing(patient_1.p_signal[: ,0])
signal_2 ,_,_= processing(patient_2.p_signal[:, 0])
signal_3 ,_,_= processing(patient_3.p_signal[:, 0])
signal_4,_,_ = processing(patient_4.p_signal[:, 0])
signal_5 ,_,_= processing(patient_5.p_signal[:, 0])
signal_6,_,_ = processing(patient_6.p_signal[:, 0])
signal_7 ,_,_= processing(patient_7.p_signal[:, 0])


print(signal_1)

time = len(signal_1)/ fs


[0.00877724 0.00944403 0.01009738 ... 0.00045572 0.00042509 0.00040158]


In [4]:
def get_Rs(signal):
    denoised_signal, y_lfiltered, window_smoothed_signal = processing(signal)
    qx, qy, sx, sy = process_signal(denoised_signal, y_lfiltered)
    qx, qy, sx, sy, Rx, Ry, qrs_on_x, qrs_on_y, qrs_off_x, qrs_off_y = process_qrs(denoised_signal, y_lfiltered, window_smoothed_signal, qx, qy, sx, sy)
    return denoised_signal,Rx

In [5]:
def non_fiducial_features_bonus(signal):
    denoised_signal,Rx = get_Rs(signal)
    nonFiducials_for_Full_signal_1 = []

    for i in range(1,len(Rx)-1):
        RR_previous = Rx[i]-Rx[i-1]
        RR_next = Rx[i+1]-Rx[i]

        nonFiducial=[]

        after_Rpeak = int (2/3*((RR_previous+RR_next)/2))

        for x in range( int(Rx[i]) , int(Rx[i])+after_Rpeak) :
            nonFiducial.append( denoised_signal[x])

        Before_Rpeak = int (1/3*((RR_previous+RR_next)/2))

        for j in range(Before_Rpeak+int(Rx[i-1]) , int(Rx[i]) ):
            nonFiducial.append( denoised_signal[j])
            
        nonFiducials_for_Full_signal_1.append(nonFiducial)
        
    list_of_non_fiducial_features_1 = []
    for i in range(len(nonFiducials_for_Full_signal_1)):
        # Define the mother wavelet
        wavelet = pywt.Wavelet('db4')

        # Define the number of levels for decomposition
        decomp_levels = 5

        # Define the list of ECG signals
        ecg_segments = np.array(nonFiducials_for_Full_signal_1[i])  # Example data

        # Decompose the signal
        decomp = pywt.wavedec(ecg_segments, wavelet, level=decomp_levels)

        CA5,CD5,CD4,CD3,CD2,CD1 =decomp 

        #only the coefficients of ECG band (1-40) use them as a feature 
        non_fiducial_feature = CA5[:40]
        non_fiducial_feature = list(non_fiducial_feature)
        if len(non_fiducial_feature) < 40:
            for i in range(40-len(non_fiducial_feature)):
                non_fiducial_feature.append(0)
        list_of_non_fiducial_features_1.append(np.array(non_fiducial_feature))
    return list_of_non_fiducial_features_1

In [6]:
non_fid1 = non_fiducial_features_bonus(signal_1)
non_fid2 = non_fiducial_features_bonus(signal_2)
non_fid3 = non_fiducial_features_bonus(signal_3)
non_fid4 = non_fiducial_features_bonus(signal_4)
non_fid5 = non_fiducial_features_bonus(signal_5)
non_fid6 = non_fiducial_features_bonus(signal_6)
non_fid7 = non_fiducial_features_bonus(signal_7)

# print(non_fid1.shape)


In [7]:
def data_frame(non_fid, label):
    df = pd.DataFrame({})
    for i in range(len(non_fid)):
        df[i] = non_fid[i]
    df = df.T
    df['label'] = label
    return df

In [8]:
B_non_fid1 = data_frame(non_fid1, 1)
B_non_fid2 = data_frame(non_fid2, 2)
B_non_fid3 = data_frame(non_fid3, 3)
B_non_fid4 = data_frame(non_fid4, 4)
B_non_fid5 = data_frame(non_fid5, 5)
B_non_fid6 = data_frame(non_fid6, 6)
B_non_fid7 = data_frame(non_fid7, 7)


In [19]:
Data = pd.concat([B_non_fid1, B_non_fid2, B_non_fid4,B_non_fid5,B_non_fid7], ignore_index=True)

In [20]:
Data

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,5.047771,5.091184,5.078735,4.967018,5.657203,3.025052,-0.591736,-0.371068,-0.333918,-0.288880,...,-0.114856,-0.107610,-0.143382,-0.222201,-0.089995,-0.064123,-0.236230,-0.540547,-0.299259,1
1,5.077009,5.120414,5.108616,4.995187,5.691347,3.041182,-0.564897,-0.362978,-0.357267,-0.285199,...,-0.121520,-0.113629,-0.147046,-0.141902,-0.192655,0.039904,-0.134352,-0.406173,-0.504051,1
2,4.978195,5.024929,5.010683,4.897055,5.599241,2.929636,-0.558846,-0.394277,-0.348331,-0.285688,...,-0.103941,-0.135927,-0.117231,-0.131336,-0.165811,0.000020,-0.019494,-0.227953,-0.579532,1
3,4.991281,5.034845,5.021516,4.913158,5.586506,3.017311,-0.523826,-0.355513,-0.319881,-0.242623,...,-0.157440,-0.190145,-0.196086,-0.234971,-0.047890,-0.083108,-0.266474,-0.584379,-0.322452,1
4,4.953668,5.001299,4.986493,4.872775,5.576415,2.900661,-0.575943,-0.376332,-0.353331,-0.253833,...,-0.141000,-0.166454,-0.189164,-0.236909,-0.052670,-0.049229,-0.234406,-0.573441,-0.366860,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,9.089901,9.166606,9.152841,8.950635,10.138785,5.659827,-0.609535,-0.947837,-0.817764,-0.610629,...,-0.266769,-0.299922,-0.369658,-0.423236,-0.308711,-0.379776,-0.740514,-0.868017,-1.436562,7
606,8.757596,8.830146,8.818165,8.622914,9.766088,5.458431,-0.629133,-0.997383,-0.946783,-0.763307,...,-0.374053,-0.348581,-0.303578,-0.336469,-0.168747,-0.075672,-0.245480,-0.629300,-0.472015,7
607,8.819978,8.898135,8.883631,8.682859,9.860479,5.424508,-0.645872,-0.984606,-0.935286,-0.779950,...,-0.196058,-0.224609,-0.250637,-0.268161,-0.316165,-0.160432,-0.150033,-0.509727,-0.866396,7
608,9.268567,9.345627,9.332158,9.124331,10.347568,5.733117,-0.718800,-0.982737,-0.865098,-0.638884,...,-0.312496,-0.272017,-0.349108,-0.411189,-0.256489,-0.265362,-0.532111,-0.803959,-1.077404,7


In [21]:
X_train = Data.drop('label', axis=1)
y = Data['label']

# Model

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [23]:
LR = LogisticRegression()
LR.fit(X_train, y)


LogisticRegression()

In [29]:
X_test = B_non_fid6.drop('label', axis=1)
y_test = B_non_fid6['label']

In [30]:
y_pred = LR.predict(X_test)
probs = LR.predict_proba(X_test)
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.0


In [31]:
threshold_percentage = 98

subject_counts = np.bincount(y_pred)
flag = 0
for subject_id, count in enumerate(subject_counts):
    if subject_id == 0:
        continue
    if count > 0:
        # Calculate the percentage of heartbeats classified to the subject
        percentage = (count / len(y_pred)) * 100

        if percentage >= threshold_percentage:
            print(f"Subject {subject_id} identified with {percentage}% certainty.")
            flag = 1
        else:
            print(f"Subject {subject_id} identified, but with {percentage}% certainty below the threshold.")
            
if flag == 0:
    print("subject is undefind")

Subject 2 identified, but with 95.8904109589041% certainty below the threshold.
Subject 5 identified, but with 4.10958904109589% certainty below the threshold.
subject is undefind


In [32]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest classifier on the training data
random_forest_classifier.fit(X_train, y)

y_pred = random_forest_classifier.predict(X_test)
probs = random_forest_classifier.predict_proba(X_test)

#Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
#print("prob:", probs)
np.max(probs)


Accuracy: 0.0


0.62

In [33]:
threshold_percentage = 98

subject_counts = np.bincount(y_pred)
flag = 0
for subject_id, count in enumerate(subject_counts):
    if subject_id == 0:
        continue
    if count > 0:
        # Calculate the percentage of heartbeats classified to the subject
        percentage = (count / len(y_pred)) * 100

        if percentage >= threshold_percentage:
            print(f"Subject {subject_id} identified with {percentage}% certainty.")
            flag = 1
        else:
            print(f"Subject {subject_id} identified, but with {percentage}% certainty below the threshold.")
            
if flag == 0:
    print("subject is undefind")

Subject 1 identified, but with 86.3013698630137% certainty below the threshold.
Subject 2 identified, but with 10.95890410958904% certainty below the threshold.
Subject 5 identified, but with 2.73972602739726% certainty below the threshold.
subject is undefind
